In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!apt-get update
!apt-get install -y cmake g++ wget


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,197 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,632 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 4,225 kB in 1s (3,664 kB/s)
Reading pack

In [4]:
!pip install dlib


In [5]:
!pip install dlib face_recognition opencv-python-headless numpy imutils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566162 sha256=96d92c2575531faefb52dda8d0268466b769a0216a4d9572809d210ced8ea49f
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


In [ ]:
import os
import cv2
import pickle
import face_recognition
import numpy as np
import imutils
import time
from datetime import datetime

# Paths for input and output
input_folder = "/content/drive/MyDrive/uploaded_folder"  # Replace with your Google Drive folder path
encodings_path = "/content/drive/MyDrive/encodings.pickle"  # Path to save encodings
output_folder = "/content/drive/MyDrive/filtered_videos"  # Dynamically created folder for unmatched videos

# Dynamically create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    print(f"[INFO] Created folder for unmatched videos: {output_folder}")

# Load or initialize encodings
if os.path.exists(encodings_path):
    print("[INFO] Loading existing encodings...")
    with open(encodings_path, "rb") as f:
        data = pickle.load(f)
else:
    print("[INFO] No encodings found. Initializing new encodings...")
    data = {"encodings": [], "names": []}

# Iterate through video files in the input folder
for video_file in os.listdir(input_folder):
    video_path = os.path.join(input_folder, video_file)

    if not video_file.endswith(('.mp4', '.avi', '.mov')):
        continue  # Skip non-video files

    print(f"[INFO] Processing video: {video_file}")
    vs = cv2.VideoCapture(video_path)

    match_found = False
    frames_without_matches = []
    start_time = time.time()  # Start timing for this video

    while True:
        ret, frame = vs.read()
        if not ret:
            break

        # Resize and convert frame to RGB
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        rgb = imutils.resize(rgb, width=750)
        r = frame.shape[1] / float(rgb.shape[1])

        # Detect faces and compute encodings using GPU (CNN model)
        boxes = face_recognition.face_locations(rgb, model="cnn")
        encodings = face_recognition.face_encodings(rgb, boxes)

        for encoding in encodings:
            matches = face_recognition.compare_faces(data["encodings"], encoding)
            distances = face_recognition.face_distance(data["encodings"], encoding)

            if len(distances) > 0 and min(distances) < 0.5:
                name = data["names"][np.argmin(distances)]
                print(f"[INFO] Match found for {name} in video: {video_file}")
                match_found = True
                break
            else:
                print(f"[INFO] No matches found for detected face in {video_file}")
                frames_without_matches.append({"frame": frame.copy(), "encoding": encoding})

        if match_found:
            break

    # Save unmatched frames as a new video if no match is found
    if not match_found and len(frames_without_matches) > 0:
        video_name = os.path.splitext(video_file)[0]
        timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
        output_video_path = os.path.join(output_folder, f"{video_name}_no_match_{timestamp}.mp4")

        frame_shape = frames_without_matches[0]["frame"].shape
        writer = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), 20,
                                 (frame_shape[1], frame_shape[0]), True)

        for frame_info in frames_without_matches:
            writer.write(frame_info["frame"])
            data["encodings"].append(frame_info["encoding"])
            data["names"].append(output_video_path)

        writer.release()
        print(f"[INFO] Saved unmatched frames to {output_video_path}")

    vs.release()

    # Save updated encodings
    with open(encodings_path, "wb") as f:
        pickle.dump(data, f)

    end_time = time.time()
    print(f"[INFO] Processed {video_file} in {end_time - start_time:.2f} seconds")


[INFO] Created folder for unmatched videos: /content/drive/MyDrive/filtered_videos
[INFO] No encodings found. Initializing new encodings...
[INFO] Processing video: faces1_Mohak_test.mp4
[INFO] No matches found for detected face in faces1_Mohak_test.mp4
[INFO] No matches found for detected face in faces1_Mohak_test.mp4
[INFO] No matches found for detected face in faces1_Mohak_test.mp4
[INFO] No matches found for detected face in faces1_Mohak_test.mp4
[INFO] No matches found for detected face in faces1_Mohak_test.mp4
[INFO] No matches found for detected face in faces1_Mohak_test.mp4
[INFO] No matches found for detected face in faces1_Mohak_test.mp4
[INFO] No matches found for detected face in faces1_Mohak_test.mp4
[INFO] No matches found for detected face in faces1_Mohak_test.mp4
[INFO] No matches found for detected face in faces1_Mohak_test.mp4
[INFO] No matches found for detected face in faces1_Mohak_test.mp4
[INFO] No matches found for detected face in faces1_Mohak_test.mp4
[INFO] No

In [1]:
# import os
# import cv2
# import pickle
# import face_recognition
# import numpy as np
# from sklearn.cluster import DBSCAN
# import time
# from datetime import datetime

# # Paths for input and output
# input_folder = "/content/drive/MyDrive/uploaded_folder"  # Replace with your Google Drive folder path
# encodings_path = "/content/drive/MyDrive/encodings.pickle"  # Path to save encodings
# output_folder = "/content/drive/MyDrive/filtered_videos"  # Dynamically created folder for unmatched videos

# # Dynamically create the output folder if it doesn't exist
# if not os.path.exists(output_folder):
#     os.makedirs(output_folder)
#     print(f"[INFO] Created folder for unmatched videos: {output_folder}")

# # Load or initialize encodings
# if os.path.exists(encodings_path):
#     print("[INFO] Loading existing encodings...")
#     with open(encodings_path, "rb") as f:
#         data = pickle.load(f)
# else:
#     print("[INFO] No encodings found. Initializing new encodings...")
#     data = []

# # Iterate through video files in the input folder
# for video_file in os.listdir(input_folder):
#     video_path = os.path.join(input_folder, video_file)

#     if not video_file.endswith(('.mp4', '.avi', '.mov')):
#         continue  # Skip non-video files

#     print(f"[INFO] Processing video: {video_file}")
#     vs = cv2.VideoCapture(video_path)

#     frame_encodings = []
#     start_time = time.time()  # Start timing for this video

#     while True:
#         ret, frame = vs.read()
#         if not ret:
#             break

#         # Resize and convert frame to RGB
#         rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#         rgb = cv2.resize(rgb, (750, int(750 * frame.shape[0] / frame.shape[1])))

#         # Detect faces and compute encodings
#         boxes = face_recognition.face_locations(rgb, model="cnn")
#         encodings = face_recognition.face_encodings(rgb, boxes)

#         for encoding in encodings:
#             frame_encodings.append(encoding)

#     vs.release()

#     # Cluster the encodings for the video
#     if frame_encodings:
#         print("[INFO] Clustering faces...")
#         clt = DBSCAN(metric="euclidean", min_samples=1)
#         clt.fit(frame_encodings)

#         unique_labels = np.unique(clt.labels_)
#         num_unique_faces = len(unique_labels)
#         print(f"[INFO] Found {num_unique_faces} unique faces in video: {video_file}")

#         if num_unique_faces == 0:
#             print(f"[INFO] No unique faces found in video: {video_file}")
#             continue

#         # Check for unmatched faces
#         unmatched = []
#         for i, label in enumerate(clt.labels_):
#             is_matched = False
#             for known_data in data:
#                 distance = np.linalg.norm(known_data["encoding"] - frame_encodings[i])
#                 if distance < 0.5:
#                     is_matched = True
#                     break
#             if not is_matched:
#                 unmatched.append((frame_encodings[i], video_file))

#         # Save unmatched video frames
#         if unmatched:
#             video_name = os.path.splitext(video_file)[0]
#             timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
#             output_video_path = os.path.join(output_folder, f"{video_name}_unmatched_{timestamp}.mp4")

#             vs = cv2.VideoCapture(video_path)
#             frame_shape = None
#             writer = None

#             for i, (encoding, file_name) in enumerate(unmatched):
#                 vs.set(cv2.CAP_PROP_POS_FRAMES, i)
#                 ret, frame = vs.read()
#                 if not ret:
#                     continue

#                 if writer is None:
#                     frame_shape = frame.shape
#                     writer = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), 20, (frame_shape[1], frame_shape[0]), True)

#                 writer.write(frame)
#                 data.append({"encoding": encoding, "video": file_name})

#             if writer:
#                 writer.release()
#             vs.release()

#             print(f"[INFO] Saved unmatched frames to {output_video_path}")

#     # Save updated encodings
#     with open(encodings_path, "wb") as f:
#         pickle.dump(data, f)

#     end_time = time.time()
#     print(f"[INFO] Processed {video_file} in {end_time - start_time:.2f} seconds")


In [10]:
import os
import cv2
import pickle
import face_recognition
import numpy as np
from datetime import datetime

# Paths for input and output
input_folder = "/content/drive/MyDrive/face_recognition_images/uploaded_image_folder"  # Replace with your Google Drive folder path
encodings_path = "/content/drive/MyDrive/face_recognition_images/encodings.pickle"  # Path to save encodings
output_folder = "/content/drive/MyDrive/face_recognition_images/filtered_images"  # Dynamically created folder for unmatched images

# Dynamically create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    print(f"[INFO] Created folder for unmatched images: {output_folder}")

# Load or initialize encodings
if os.path.exists(encodings_path):
    print("[INFO] Loading existing encodings...")
    with open(encodings_path, "rb") as f:
        data = pickle.load(f)
else:
    print("[INFO] No encodings found. Initializing new encodings...")
    data = []

# Iterate through image files in the input folder
for image_file in os.listdir(input_folder):
    image_path = os.path.join(input_folder, image_file)

    if not image_file.lower().endswith(('.png', '.jpg', '.jpeg')):
        continue  # Skip non-image files

    print(f"[INFO] Processing image: {image_file}")

    # Load and preprocess the image
    image = cv2.imread(image_path)
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Detect faces and compute encodings
    boxes = face_recognition.face_locations(rgb, model="cnn")
    encodings = face_recognition.face_encodings(rgb, boxes)

    if not encodings:
        print(f"[INFO] No faces detected in image: {image_file}")
        continue

    unmatched = []
    for encoding in encodings:
        is_matched = False
        for known_data in data:
            distance = np.linalg.norm(known_data["encoding"] - encoding)
            if distance < 0.5:
                is_matched = True
                break
        if not is_matched:
            unmatched.append(encoding)

    # Save unmatched images
    if unmatched:
        output_image_path = os.path.join(output_folder, f"unmatched_{datetime.now().strftime('%Y%m%d%H%M%S')}_{image_file}")
        cv2.imwrite(output_image_path, image)
        print(f"[INFO] Saved unmatched image: {output_image_path}")

        # Add unmatched encodings to the database
        for encoding in unmatched:
            data.append({"encoding": encoding, "image": image_file})

# Save updated encodings
with open(encodings_path, "wb") as f:
    pickle.dump(data, f)

print("[INFO] Processing completed for all images.")


[INFO] Created folder for unmatched images: /content/drive/MyDrive/face_recognition_images/filtered_images
[INFO] No encodings found. Initializing new encodings...
[INFO] Processing image: emma_mkey_1.png
[INFO] Saved unmatched image: /content/drive/MyDrive/face_recognition_images/filtered_images/unmatched_20250105012136_emma_mkey_1.png
[INFO] Processing image: emma_mc key_2.png
[INFO] Processing image: margot_robbie.png
[INFO] Saved unmatched image: /content/drive/MyDrive/face_recognition_images/filtered_images/unmatched_20250105012136_margot_robbie.png
[INFO] Processing image: sg3.png
[INFO] Saved unmatched image: /content/drive/MyDrive/face_recognition_images/filtered_images/unmatched_20250105012136_sg3.png
[INFO] Processing image: salman_khan_1.jpg
[INFO] Saved unmatched image: /content/drive/MyDrive/face_recognition_images/filtered_images/unmatched_20250105012137_salman_khan_1.jpg
[INFO] Processing image: salman_khan_2.png
[INFO] Processing completed for all images.
